<a href="https://colab.research.google.com/github/kishanthan-s/Agenda_Tracker/blob/main/Chatbot_latest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers faiss-cpu gradio pypdf sentencepiece
!pip install langchain_community
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.0 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
The token `csv` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushin

In [3]:
from langchain import HuggingFacePipeline

from transformers import AutoTokenizer

from langchain.embeddings import HuggingFaceEmbeddings

from langchain.document_loaders.csv_loader import CSVLoader

from langchain.vectorstores import FAISS

from langchain.chains import RetrievalQA

import transformers

import torch

import gradio

import textwrap

from PyPDF2 import PdfReader, PdfWriter

from langchain.docstore.document import Document

import textwrap


In [4]:
# Define the model
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# # Create the pipeline for text generation
pipeline = transformers.pipeline(
    "text-generation",  # Task
    model=model_name,  # Model name
    tokenizer=tokenizer,  # Tokenizer
    torch_dtype=torch.bfloat16,  # Use bfloat16 for efficiency
    trust_remote_code=True,  # Allow remote code if necessary
    device_map="auto",  # Automatically place model on the appropriate device
    max_length=1000,  # Maximum length of generated text
    do_sample=True,  # Enable sampling
    top_k=10,  # Use top-k sampling
    num_return_sequences=1,  # Return a single sequence
    eos_token_id=tokenizer.eos_token_id,  # End of sequence token ID
    truncation=True
)


llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-4-473846d62055>:24: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


In [5]:
pdf_contet='/content/modified-1-3.pdf'

from PyPDF2 import PdfReader, PdfWriter

input_pdf = pdf_contet  # Replace with your actual PDF content variable
reader = PdfReader(input_pdf)

# Fixed length (number of pages per split)
length = 1

# Split PDF into chunks
chunks = [reader.pages[i:i + length] for i in range(0, len(reader.pages), length)]

doc = []
doc_for_llm = []

# Extract and print the content of each chunk
for idx, start_idx in enumerate(range(0, len(reader.pages), length)):
    # print(f"Chunk {idx + 1}:")
    chunk_content = ""
    for page_num in range(start_idx, min(start_idx + length, len(reader.pages))):
        chunk_content += reader.pages[page_num].extract_text()
    doc.append(chunk_content)

    # Create a dictionary for LLM processing with content and metadata
    i = {
        "page_content": chunk_content,
        "metadata": {
            "page": idx + 1  # Use idx + 1 for human-readable page numbering
        }
    }
    doc_for_llm.append(i)

# Now, doc_for_llm contains the desired structure


In [6]:
print(doc_for_llm[0])

{'page_content': "Error Code:   118010143  \nDescription:  The currencies of fees rated on one service are different.  \nPossible Causes:  The currencies of fees rated on one service are different  \nProcedure:  Check the subscriber's tariff offering and adjust the default currency or tariff conditions to \nprevent fees in different currencies within the same session . \nError Code:   118010144  \nDescription:  Inconsistent service usage type.  \nPossible Causes:  Conversion failed due to inconsistent service usage type.  \nProcedure:  Cancel the operati on.  \nError Code:  118010145  \nDescription:  Session timed out.  \nPossible Causes:  Session timed out.  \nProcedure:  Check sessions where the OCG or PCRF stops reporting messages after timeout.  \nErro r Code:  18010147  \nDescri ption:  Receiving messages time out.  \nPossible Causes:  Receiving messages time out.  \nProcedure:  Locate the fault according to logs or resend the message.  \nError Code:  118010149  \nDescription:  Th

In [7]:

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


<ipython-input-7-83794808db26>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Extract the content from all_documents
document_texts = [doc["page_content"] for doc in doc_for_llm]

# Embed the documents using the embeddings
document_embeddings = embeddings.embed_documents(document_texts)

# Pair document texts with their embeddings
text_embedding_pairs = list(zip(document_texts, document_embeddings))

# Create the FAISS index with the document embeddings
vectorstore = FAISS.from_embeddings(text_embedding_pairs, embeddings)

# Create the retriever from the vector store, setting the number of results to return in search
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})


In [9]:
vectorstore.save_local("/content/faiss_index_path")

In [10]:
vectorstore = FAISS.load_local(
    "/content/faiss_index_path",
    embeddings,
    allow_dangerous_deserialization=True
)


In [11]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    return_source_documents=True,
    retriever=retriever,
)

In [12]:
query = "What is the does 11801 mean"

try:
    result = chain(query)
    wrapped_text = f"Answer: {result['result']}"
    print("Result:", wrapped_text)
except Exception as e:
    print("Error during query execution:", str(e))


<ipython-input-12-6c414c2da875>:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain(query)


Result: Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Procedure:  Increase the balance by recharge or transfer.  
Error Code:  118010203  
Description:  The service does not support multiple modifications of the bill cycle.  
Possible Causes:  The service does not support multiple modifications of the bill cycle.  
Procedure:  Do not modify the bill cycle for multiple times.  
Error Code:  118010206 Description:   
Incorrect fields in the management message.  
Possible Causes:  The management message has invalid fields, like an out -of-range Optype during 
offering switchover.  
Procedure:  Modify the incorrect fields and send the management message again.  
Error Code:  118010207  
Description:  Unknown service number type.  
Possible Causes:  The system cannot identify the normalized number type.  
Procedure:  Change the t ype of the number to be analyzed and 